# Models + Evaluation Metrics

- **Goal:** Prediction Recognition

- **Purpose:** To train our models and to make predictions on unseen data.

- **Misc:**
    - `%store`: Cell magic will store the variable of interest so we can load in another notebook

In [1]:
import os
import sys

import pandas as pd

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

from pipelines import BasePipeline
from data_processing import DataProcessing
from classification_models import PerceptronModel

In [2]:
%store -r shuffled_df
%store -r tfidf_vectorized_features
%store -r tfidf_vectorized_features_df
%store -r encoded_word_level_pos_ner_df

pd.set_option('max_colwidth', 800)
pre_processed_df = shuffled_df.copy()
pre_processed_df

,Base Sentence,Prediction Label,Model Name,Domain,Template Number
0,the financial report shows a significant increase in revenue last year.,0,llama-3.3-70b-versatile,any,NaN
1,"in 2025-08-20, the number of investors in the new york stock exchange will likely have a 20% increase, as predicted by policy expert, david lee, on 20 october 2024.",1,llama-3.3-70b-versatile,policy,5.0
2,the research team is conducting experiments to gather more data information.,0,llama-3.3-70b-versatile,any,NaN
3,"on 2024-10-15, dr. thompson, a health expert from the world health organization, predicts that the obesity rate at the united states will likely decrease by 3% in q2 of 2026.",1,llama-3.3-70b-versatile,health,1.0
4,"james davis, a financial expert, predicts on 2024/08/22 that the revenue at visa may rise by 15% to $25 billion in 2028.",1,llama-3.3-70b-versatile,financial,3.0
...,...,...,...,...,...
75,"on 2024-10-15, policy expert, rachel kim, predicts that the stock price at goldman sachs will rise by 15% in q2 of 2026.",1,llama-3.3-70b-versatile,policy,1.0
76,"on 2024-10-15, rachel patel, a financial analyst, predicts that the operating cash flow at general motors will likely decrease by $5 billion to $10 billion in q2 of 2026.",1,llama-3.3-70b-versatile,financial,1.0
77,the manager is reviewing the quarterly sales report carefully now.,0,llama-3.3-70b-versatile,any,NaN
78,the customer service department is receiving a high volume of calls.,0,llama-3.3-70b-versatile,any,NaN


In [3]:
base_pipeline = BasePipeline()

## Combine Features (TF x IDF and POS & NER Encodings)

In [4]:
all_features_df = DataProcessing.concat_dfs([tfidf_vectorized_features_df, encoded_word_level_pos_ner_df], axis=1)
all_features_df

,0,1,2,3,4,5,6,7,8,9,...,114,115,116,117,118,119,120,121,122,123
0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0.193189,0.000000,0.0,0.000000,0.564387,0.118179,0.218296,0.000000,0.0,...,0,1,0,1,1,0,0,0,0,0
2,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0.000000,0.173629,0.0,0.185392,0.000000,0.125288,0.000000,0.210604,0.0,...,1,1,0,1,1,0,0,0,0,0
4,0.0,0.228784,0.000000,0.0,0.207093,0.000000,0.139954,0.000000,0.000000,0.0,...,0,0,0,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.0,0.000000,0.196231,0.0,0.419052,0.000000,0.141598,0.000000,0.238020,0.0,...,0,1,0,1,1,0,0,0,0,0
76,0.0,0.000000,0.347163,0.0,0.185342,0.000000,0.125255,0.000000,0.210547,0.0,...,0,1,0,0,1,0,0,0,0,0
77,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
78,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0


# Split Features and Prediction Labels

In [5]:
prediction_labels = shuffled_df['Prediction Label']
prediction_labels

0     0
1     1
2     0
3     1
4     1
     ..
75    1
76    1
77    0
78    0
79    0
Name: Prediction Label, Length: 80, dtype: int64

## Models

1. Perceptron
    - $ x_n \in X $, `tfidf_vectorized_features`
        - N, `tfidf_vectorized_features_n`. Each row (formally document).
        - D, `tfidf_vectorized_features_d`. Each column (formally unique terms/features)
        - Thus, $ X \in R^{N \times D} $
    - $ w^T $, Weights, which are randomly initialize (in sklearn)
        - N, `tfidf_vectorized_features_d`. Each column (formally unique terms/features)
        - D, 1
        - Thus, $ w^T \in R^{N \times D} $
    
    $$
    (w^T \cdot x_n) \Rightarrow (100 \times 1) \cdot (1 \times 100)
    $$

In [6]:
# features_N, features_D = all_features_df.shape[0], all_features_df.shape[1]
# print(f"There are {features_N} features (rows) with {features_D} words (columns).")

In [7]:
prediction_labels = pre_processed_df['Prediction Label']
X_train, X_test, y_train, y_test = DataProcessing.split_data(all_features_df, prediction_labels)

In [8]:
perception_model = PerceptronModel()

perception_model.train_model(X_train, y_train)
y_predictions = perception_model.predict(X_test)
y_predictions

0     1
1     0
2     1
3     1
4     0
5     1
6     1
7     1
8     1
9     1
10    0
11    1
12    1
13    1
14    0
15    0
dtype: int64

In [9]:
DataProcessing.join_predictions_with_labels(pre_processed_df, y_test, y_predictions, perception_model)

,Base Sentence,Model Name,Domain,Template Number,Prediction Label,Perceptron Model Prediction
64,the manager is reviewing the sales report from last quarter carefully.,llama-3.3-70b-versatile,any,NaN,0,1
65,the research team is conducting experiments in the laboratory.,llama-3.3-70b-versatile,any,NaN,0,0
66,"policy analyst, michael chen, predicts on 21 august 2024 that the gdp growth rate in the united states may stay stable at 3% in 2027.",llama-3.3-70b-versatile,policy,3.0,1,1
67,"in 2025-01-01, david taylor from the weather underground predicts that the cloud cover in miami will decrease by 15% in 2026-06-01.",llama-3.3-70b-versatile,weather,2.0,1,1
68,the ceo is giving a presentation to the board of directors.,llama-3.3-70b-versatile,any,NaN,0,0
69,"in 2028-02-20, the wind speed in chicago has a probability of 20% to decrease by 10 mph, as predicted by james lee, a weather analyst from the meteorological department, on 2024-06-12.",llama-3.3-70b-versatile,weather,5.0,1,1
70,"emily chen, a weather expert from the bureau of meteorology, predicts on 2024-11-10 that the temperature in sydney will fall by 8°c in 2025-07-01.",llama-3.3-70b-versatile,weather,3.0,1,1
71,the marketing department is creating a new advertising campaign strategy.,llama-3.3-70b-versatile,any,NaN,0,1
72,the company is currently undergoing a major restructuring effort to improve efficiency.,llama-3.3-70b-versatile,any,NaN,0,1
73,"policy expert, mark davis, predicts on 15 october 2024 that the revenue growth rate at citigroup will increase by 10% in 2028.",llama-3.3-70b-versatile,policy,3.0,1,1


In [10]:
metrics = base_pipeline.evaluation_metrics(y_test, y_predictions, default_metrics=True)
metrics

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00        11

    accuracy                           1.00        16
   macro avg       1.00      1.00      1.00        16
weighted avg       1.00      1.00      1.00        16



In [11]:
metrics = base_pipeline.evaluation_metrics(y_test, y_predictions, default_metrics=False)
metrics

{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'F1 Score': 1.0}